In [ ]:
import numpy as np
import tensorflow as tf

# %matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as colors 

import ipysh
import Hunch_utils  as Htls
import Hunch_lsplot as Hplt
import Hunch_tSNEplot as Hsne

# Load the TensorBoard notebook extension
%load_ext tensorboard

%aimport models.AEFIT2
%aimport models.AEFIT3
%aimport models.base

!pip install livelossplot --user


In [ ]:
qsh = Htls.QSH_Dataset()
import os
file = ipysh.abs_builddir+'/te_db_r15.npy'
if os.path.isfile(file):
    qsh.load(file)
else:
    qsh.load(ipysh.abs_builddir+'/te_db_1.npy')    
    qsh.rebalance_prel(15)
    qsh.save(ipysh.abs_builddir+'/te_db_r15.npy')
    
qsh.shuffle()
# qsh.filter_number_set(15)
qsh.clean_up_poorcurves(5)
qsh.dim = 15
qsh.set_null(np.nan)
qsh.set_normal_positive()
qsh.unbias_mean(0.5, 'te')
qsh.set_normal_positive()

print("QSH rebalanced 15 points size: ", len(qsh))

qsh.set_null(np.nan)
qsh.clip_values(0.1,0.6)
qsh.set_normal_positive()

## VAE AEFIT3 reconstruction - beta

This is the Analytical way to compute KL term ... beta is applied to KL

In [ ]:
vae = models.AEFIT3.AEFIT3(latent_dim=2, feature_dim=30, dprate=0., scale=4, beta=1.)

In [ ]:
p = Hplt.LSPlotBokeh()
p.set_model(vae)
p.set_data(qsh, counts=3000)
p.plot_notebook(notebook_url='http://172.17.0.2:8888')

In [ ]:
# models.base.train(vae, qsh, batch=200, epoch=1, loss_factor=1e-3)

In [ ]:
thread = models.base.train_thread(vae, qsh, batch=200, epoch=5, learning_rate=1e-3)
thread.control_panel()

In [ ]:
vae.save('step6.5_r15_v3')

In [ ]:
vae.load('step6.6_r15_v3')

In [ ]:
qsh_pos = 0

In [ ]:
fig = plt.figure('test_curve',figsize=(18, 6))
plt.clf()
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)    
# ax1.set_xlim(-2.,2.)
ax2.set_ylim(0.,1.)


for xy in qsh.ds_array.skip(qsh_pos).take(1):    
    xy,_ = xy
    x,y = tf.split(xy,2)
    ax2.scatter(x,y,s=80)
    m,v  = vae.encode([xy], training=False)
    gpt = m[0].numpy()
    ax1.scatter(gpt[0],gpt[1])
    
    XY = vae.decode(m,apply_sigmoid=True, training=False)
    X,Y = tf.split(XY[0], 2)
    X,Y = (X.numpy(), Y.numpy())
    ax2.scatter(X,Y,s=40)

print(qsh_pos)    
qsh_pos += 1
    